In [2]:
import sys
from functools import reduce
import numpy as np
import cirq
from stabilizer_states import StabilizerStates
from stabilizer_toolkit.decompositions import rank2, validate_decompositions
from stabilizer_toolkit.magic_states import enumerate_ccz
from stabilizer_toolkit.helpers.unitary import get_tensored_unitary

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
np.set_printoptions(precision=3, linewidth=sys.maxsize, edgeitems=4, threshold=1024, suppress=True) 

For five qubit $\mathtt{CCZ}$ magic states, we already know we can search within the real stabilizer states, reducing the dataset down from $2,423,520$ complex-valued five qubit stabilizer states down to only $146,880$ real-valued stabilizer states.

In [5]:
StabilizerStates.count(5)

2423520

Furthermore, we can load a pre-processed ternary form of all the real stabilizer states, which is much less data to load.

In [22]:
S5 = StabilizerStates(5, 'ternary')
print(len(S5))

146880


If we were to use brute-force search over all the real stabilizer states, then this takes over a full day to find rank-2 decompositions of a **single**, distinct five qubit $\mathtt{CCZ}$ magic state. Running the next cell is not recommended!

In [ ]:
_, state, _, _ = next(enumerate_ccz(5))
decompositions, coeffs = rank2.search_all_stabilizer_states(state, S5, num_cpus=8)

146880
[19] [1 0 0 1 1]
[11] [0 1 0 1 1]
[7] [0 0 1 1 1]


2023-04-20 23:47:32,540	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
 95%|█████████████████████████████████████████████████████▏  | 10254043741/10786793760 [105:58:06<8:43:32, 16960.04it/s]

For comparison, if we use the ternary search method, then it only takes roughly ~6 mins (on my Quad-Core Intel i7 laptop) to find the $15$ rank-2 decompositions for the same state.

In [23]:
_, state, _, _ = next(enumerate_ccz(5))
decompositions, coeffs = rank2.ternary_search(state, S5)

2023-05-11 22:58:33,488	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
100%|██████████████████████████████████████████████████████████████████████████| 146880/146880 [09:56<00:00, 246.37it/s]


In [24]:
validate_decompositions(state, decompositions, coeffs)

15 decompositions
|ψ〉	= [ 0.177  0.177  0.177  0.177  0.177  0.177  0.177 -0.177  0.177  0.177  0.177 -0.177  0.177  0.177  0.177  0.177  0.177  0.177  0.177 -0.177  0.177  0.177  0.177  0.177  0.177  0.177  0.177  0.177  0.177  0.177  0.177 -0.177]

✅	= [0.707] * [ 0.    0.    0.25  0.25  0.    0.    0.25 -0.25  0.    0.    0.25 -0.25  0.    0.    0.25  0.25  0.    0.    0.25 -0.25  0.    0.    0.25  0.25  0.    0.    0.25  0.25  0.    0.    0.25 -0.25]
	+ [0.707] * [0.25 0.25 0.   0.   0.25 0.25 0.   0.   0.25 0.25 0.   0.   0.25 0.25 0.   0.   0.25 0.25 0.   0.   0.25 0.25 0.   0.   0.25 0.25 0.   0.   0.25 0.25 0.   0.  ]

✅	= [0.707] * [ 0.    0.25  0.25  0.    0.25  0.    0.   -0.25  0.25  0.    0.   -0.25  0.    0.25  0.25  0.    0.25  0.    0.   -0.25  0.    0.25  0.25  0.    0.    0.25  0.25  0.    0.25  0.    0.   -0.25]
	+ [0.707] * [0.25 0.   0.   0.25 0.   0.25 0.25 0.   0.   0.25 0.25 0.   0.25 0.   0.   0.25 0.   0.25 0.25 0.   0.25 0.   0.   0.25 0.25 0.   0.   0.25 0. 

True

There are $29$ distinct $\mathtt{CCZ}$ circuits and corresponding magic states, so at roughly ~10 mins per search it will take roughly 4.8 hours. You can run the next cell to perform a rank-2 decomposition search for each of these magic states. Observe that for states where $\mathtt{CCZ}$s overlap on two qubits in the corresponding circuit, then this reduces down to a single $\mathtt{CCZ}$ in the circuit and there are subsequently $15$ rank-2 decompositions. For states where $\mathtt{CCZ}$s overlap only on a single qubit there is only a _single_ rank-2 decomposition.

In [21]:
for index, state, D, circuit in enumerate_ccz(5):
    decompositions, coeffs = rank2.ternary_search(state, S5)
    print()
    print(f"Distinct circuit index {index}")
    print(f"|ψ〉= {state}")
    print(f" D = diag({np.diag(D)})")
    print(circuit)
    valid = validate_decompositions(state, decompositions, coeffs, show=False)
    status = "✅" if valid else "❌"
    print(f"All {len(decompositions)} decomposition(s) rank-2: {status}")
    print()

2023-05-10 22:14:39,187	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
100%|██████████████████████████████████████████████████████████████████████████| 146880/146880 [09:21<00:00, 261.69it/s]



Distinct circuit index 19
|ψ〉= [ 0.177  0.177  0.177  0.177  0.177  0.177  0.177 -0.177  0.177  0.177  0.177 -0.177  0.177  0.177  0.177  0.177  0.177  0.177  0.177 -0.177  0.177  0.177  0.177  0.177  0.177  0.177  0.177  0.177  0.177  0.177  0.177 -0.177]
 D = diag([ 1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.])
[[1 0 0]
 [0 1 0]
 [0 0 1]
 [1 1 1]
 [1 1 1]]
All 15 decomposition(s) rank-2: ✅



2023-05-10 22:24:08,710	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
100%|██████████████████████████████████████████████████████████████████████████| 146880/146880 [09:29<00:00, 257.89it/s]



Distinct circuit index 20
|ψ〉= [ 0.177  0.177  0.177  0.177  0.177  0.177  0.177  0.177  0.177  0.177  0.177  0.177  0.177 -0.177  0.177 -0.177  0.177  0.177  0.177 -0.177  0.177  0.177  0.177 -0.177  0.177  0.177  0.177 -0.177  0.177 -0.177  0.177  0.177]
 D = diag([ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1. -1.  1.  1.  1. -1.  1.  1.  1. -1.  1.  1.  1. -1.  1. -1.  1.  1.])
[[1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 1]]
All 1 decomposition(s) rank-2: ✅



2023-05-10 22:33:47,609	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
100%|██████████████████████████████████████████████████████████████████████████| 146880/146880 [09:32<00:00, 256.59it/s]


...


Distinct circuit index 495
|ψ〉= [ 0.177  0.177  0.177  0.177  0.177  0.177  0.177 -0.177  0.177  0.177  0.177 -0.177  0.177 -0.177 -0.177  0.177  0.177  0.177  0.177  0.177  0.177 -0.177 -0.177 -0.177  0.177 -0.177 -0.177 -0.177  0.177 -0.177 -0.177  0.177]
 D = diag([ 1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1. -1.  1. -1. -1.  1.  1.  1.  1.  1.  1. -1. -1. -1.  1. -1. -1. -1.  1. -1. -1.  1.])
[[1 1 1 1 0 0 0 0]
 [1 1 0 0 1 1 1 0]
 [0 0 1 1 1 1 0 1]
 [1 0 1 0 1 0 1 1]
 [0 1 0 1 0 1 1 1]]
All 15 decomposition(s) rank-2: ✅



2023-05-11 02:17:05,435	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
100%|██████████████████████████████████████████████████████████████████████████| 146880/146880 [08:40<00:00, 282.21it/s]



Distinct circuit index 511
|ψ〉= [ 0.177  0.177  0.177  0.177  0.177  0.177  0.177 -0.177  0.177  0.177  0.177 -0.177  0.177 -0.177 -0.177  0.177  0.177  0.177  0.177 -0.177  0.177 -0.177 -0.177  0.177  0.177 -0.177 -0.177  0.177  0.177 -0.177 -0.177 -0.177]
 D = diag([ 1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1. -1.  1. -1. -1.  1.  1.  1.  1. -1.  1. -1. -1.  1.  1. -1. -1.  1.  1. -1. -1. -1.])
[[1 1 1 1 1 0 0 0 0]
 [1 1 0 0 0 1 1 1 0]
 [0 0 1 1 0 1 1 0 1]
 [1 0 1 0 1 1 0 1 1]
 [0 1 0 1 1 0 1 1 1]]
All 1 decomposition(s) rank-2: ✅



2023-05-11 02:25:56,106	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
100%|██████████████████████████████████████████████████████████████████████████| 146880/146880 [08:36<00:00, 284.53it/s]



Distinct circuit index 1023
|ψ〉= [ 0.177  0.177  0.177  0.177  0.177  0.177  0.177 -0.177  0.177  0.177  0.177 -0.177  0.177 -0.177 -0.177  0.177  0.177  0.177  0.177 -0.177  0.177 -0.177 -0.177  0.177  0.177 -0.177 -0.177  0.177 -0.177  0.177  0.177  0.177]
 D = diag([ 1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1. -1.  1. -1. -1.  1.  1.  1.  1. -1.  1. -1. -1.  1.  1. -1. -1.  1. -1.  1.  1.  1.])
[[1 1 1 1 1 1 0 0 0 0]
 [1 1 1 0 0 0 1 1 1 0]
 [1 0 0 1 1 0 1 1 0 1]
 [0 1 0 1 0 1 1 0 1 1]
 [0 0 1 0 1 1 0 1 1 1]]
All 1 decomposition(s) rank-2: ✅

